# Create a boxplot of avoided fuel emissions distribution 

Attribute: 'emissions_avoided'

Filters:
* Sector = Food Manufacturing (NAICS code = 311*)
* Period = 2014 - 2024 (last 10 years)
* Implemented vs Not Implemented
* ARCs = 2.1224 (Replace Boiler)
* State = AZ, CA
    * Question: do we want to offer a comparative boxplots? or only aggregate?

### Notebook generates the following boxplot:

In [ ]:
from IPython.display import Image

image_path = '../assets/emissions_avoided_facetgrid.png'

# Display the image
Image(image_path)

# Data Preparation

In [ ]:
# Import libraries
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import janitor
from janitor import clean_names


In [ ]:
# Import datasets

# ------- define paths -------
relative_path = Path('../data/intermediate_data/') # define relative path
absolute_path = relative_path.resolve() # get absolute path


# ------- import data -------
assess_df = pd.read_csv(absolute_path/'iac_assess_tidy.csv') # import IAC assess dataset
ec_emissions_df = pd.read_csv(absolute_path/'emissions_tidy.csv') # import emissions dataset
ec_generation_df = pd.read_csv(absolute_path/'generation.csv') # import electricity generation dataset
recc_integrated_ppi_df = pd.read_csv(absolute_path/'recc_integrated_ppi.csv') # import an integrated recc dataset with adjusted impcost
fuel_emission_factors_df = pd.read_excel(absolute_path/'emission_factors_tidy.xlsx', sheet_name='Sheet1') # import fuel emission factors

In [ ]:
ec_emissions_df

In [ ]:
fuel_emission_factors_df

In [ ]:
assess_df = assess_df.clean_names()

In [ ]:
recc_integrated_ppi_df = recc_integrated_ppi_df.dropna(subset=['sourccode', 'conserved','sourconsv','saved'], how='all')
recc_integrated_ppi_df[recc_integrated_ppi_df['superid']=='AM043901']

In [ ]:
# Add Sector and State attributes to recc_integrated_ppi_df from assess_df
integrated_ppi_df = pd.merge(recc_integrated_ppi_df, assess_df[['state','naics','id']],
                                  on='id',
                                  how='left')

integrated_ppi_df.drop_duplicates(inplace = True)

In [ ]:
# check unique power source codes 
integrated_ppi_df['sourccode'].unique()


In [ ]:
integrated_ppi_df[integrated_ppi_df['superid']=='WV061012']

#### Merge Fuel Emission Factors into the integrated recc table

In [ ]:
# add fuel emission factors to the integrated recc df
integrated_df = pd.merge(integrated_ppi_df, fuel_emission_factors_df[['sourccode','emission_type','emission_factor','emission_factor_units']],
                                  on='sourccode',
                                  how='left')

In [ ]:
integrated_df[integrated_df['superid'].isin(['WV061012', 'AM057403'])]

#### Calculate fuel emission factors

In [ ]:
# Calculate fuel emissions avoided
integrated_df['emissions_avoided'] = integrated_df['emission_factor'] * integrated_df['conserved']

In [ ]:
integrated_df[integrated_df['superid'].isin(['WV061012', 'AM057403'])]

# Merge electricity emissions into an integrated recc table

In [ ]:
ec_emissions_df = ec_emissions_df[(ec_emissions_df['producer_type']=='Total Electric Power Industry')& # units = metric ton
                                  (ec_emissions_df['energy_source']=='All Sources')]

ec_generation_df = ec_generation_df[(ec_generation_df['type_of_producer']=='Total Electric Power Industry')&
                                  (ec_generation_df['energy_source']=='Total')]

In [ ]:
ec_emissions_df

In [ ]:
# calculate emission factors
# Total Emissions/Total Electricity Generated
ec_emission_factors_df = pd.merge(ec_generation_df,ec_emissions_df[['year','state','emission_type','amount']])
ec_emission_factors_df['emission_factor'] = ec_emission_factors_df['amount'] / ec_emission_factors_df['generation_megawatthours_']

# add column emission_factor_units
ec_emission_factors_df['emission_factor_units'] = 'kg/kWh'
ec_emission_factors_df['sourccode'] = 'EC'

In [ ]:
ec_emission_factors_df

In [ ]:
integrated_df.columns

In [ ]:
# combine ec_emission_factors_df with the integrated recc table
integrated_df = pd.merge(integrated_df, ec_emission_factors_df[['state','year','emission_type','emission_factor','emission_factor_units','sourccode']],
                         left_on=['fy','state','sourccode'],
                         right_on=['year','state','sourccode'],
                         how='left')

# merge overlapping columns
integrated_df['emission_type'] = integrated_df['emission_type_x'].combine_first(integrated_df['emission_type_y'])
integrated_df['emission_factor_units'] = integrated_df['emission_factor_units_x'].combine_first(integrated_df['emission_factor_units_y'])
integrated_df['emission_factor'] = integrated_df['emission_factor_x'].combine_first(integrated_df['emission_factor_y'])

# drop the old duplicate columns
integrated_df.drop(columns=['emission_type_x', 'emission_type_y', 'emission_factor_units_x', 'emission_factor_units_y', 'year','emission_factor_x','emission_factor_y'], 
                   inplace=True)


In [ ]:
integrated_df[integrated_df['superid'].isin(['WV061012', 'AM057403'])]

In [ ]:
integrated_df.loc[integrated_df['sourccode'] == 'EC', 'emissions_avoided'] = (
    integrated_df['emission_factor'] * integrated_df['conserved']
)
integrated_df.loc[integrated_df['sourccode'] == 'EC', 'emissions_avoided'] = (
    integrated_df['emission_factor'] * integrated_df['conserved']
)


In [ ]:
integrated_df[integrated_df['id'].isin(['SF0532', 'OR0712', 'MI0415','IC0115'])]

In [ ]:
integrated_df[integrated_df['superid'].isin(['WV061012', 'AM057403','MI041503','SF053206','SF053207','MI041503','IC011501'])]

### Set Filters

In [ ]:
# get arcs2 for sector = 311 (food production)
recc_integrated_ppi_311_df = integrated_ppi_df[integrated_ppi_df['naics'].astype(str).str.startswith('311')]
recc_integrated_ppi_311_df = integrated_ppi_df[integrated_ppi_df['arc2'].astype(str).str.startswith('2.')]

recc_integrated_ppi_311_df[recc_integrated_ppi_311_df['superid']=='AM043901']
recc_integrated_ppi_311_df[recc_integrated_ppi_311_df['id']=='AM0439']

recc_integrated_ppi_311_df['arc2'].unique()[:15]

In [ ]:
# set filters
arc2_filter = [2.1224]
sector_filter = '311'
period_filter_from = 2010
period_filter_to = 2024

In [ ]:
filtered_df = integrated_df[
    (integrated_df['naics'].astype(str).str.startswith(sector_filter)) &
    (integrated_df['arc2'].isin(arc2_filter)) &
    (integrated_df['fy'] >= period_filter_from) &
    (integrated_df['fy'] <= period_filter_to) &
    (integrated_df['state'].isin(['AZ','CA']))&
    (integrated_df['source_rank']=='PSOURCCODE')	
]

filtered_EC_df = integrated_df[
    #(integrated_df['naics'].astype(str).str.startswith(sector_filter)) &
    (integrated_df['arc2'].isin(arc2_filter)) &
    #(integrated_df['fy'] >= period_filter_from) &
    #(integrated_df['fy'] <= period_filter_to) &
    #(integrated_df['state'].isin(['AZ','CA']))&
    (integrated_df['source_rank']=='PSOURCCODE') &
    (integrated_df['emission_factor_units']=='kg/kWh')	
]

filtered_other_fuels_df = integrated_df[
    (integrated_df['naics'].astype(str).str.startswith(sector_filter)) &
    (integrated_df['arc2'].isin(arc2_filter)) &
    (integrated_df['fy'] >= period_filter_from) &
    (integrated_df['fy'] <= period_filter_to) &
    #(integrated_df['state'].isin(['AZ','CA']))&
    (integrated_df['source_rank']=='PSOURCCODE') &
    (integrated_df['emission_factor_units']=='kg/MMBtu') 	
]

filtered_CO2_df = integrated_df[
    (integrated_df['naics'].astype(str).str.startswith(sector_filter)) &
    (integrated_df['arc2'].isin(arc2_filter)) &
    (integrated_df['fy'] >= period_filter_from) &
    (integrated_df['fy'] <= period_filter_to) &
    (integrated_df['emission_type'].isin(['CO2']))&
    (integrated_df['state'].isin(['AZ','CA']))&
    (integrated_df['source_rank']=='PSOURCCODE')
]

filtered_SO2_df = integrated_df[
    (integrated_df['naics'].astype(str).str.startswith(sector_filter)) &
    (integrated_df['arc2'].isin(arc2_filter)) &
    (integrated_df['fy'] >= period_filter_from) &
    (integrated_df['fy'] <= period_filter_to) &
    (integrated_df['emission_type'].isin(['SO2']))&
    (integrated_df['state'].isin(['AZ','CA']))&
    (integrated_df['source_rank']=='PSOURCCODE')
]

filtered_NOx_df = integrated_df[
    (integrated_df['naics'].astype(str).str.startswith(sector_filter)) &
    (integrated_df['arc2'].isin(arc2_filter)) &
    (integrated_df['fy'] >= period_filter_from) &
    (integrated_df['fy'] <= period_filter_to) &
    (integrated_df['emission_type'].isin(['NOx']))&
    (integrated_df['state'].isin(['AZ','CA']))&
    (integrated_df['source_rank']=='PSOURCCODE')
]

In [ ]:
filtered_df

In [ ]:
filtered_CO2_df.columns

In [ ]:
# test that filters values are correct
print("Unique values in arc2:", filtered_CO2_df['arc2'].unique())
print("Unique values in arc2:", filtered_CO2_df['fy'].unique())
print(len(filtered_CO2_df['arc2'].unique()))

filtered_CO2_df

Managing Outliers for Better Visualization 
Method: set showfliers=False in the sns.boxplot. This allows keeping all data in a dataframe while hiding the outliers on a chart.
Seaborn uses Interquartile Range (IQR) Rule to determine outliers:

Outlier Calculation Using IQR:

First Quartile (Q1) = 25th percentile
Third Quartile (Q3) = 75th percentile
Interquartile Range (IQR) = Q3 - Q1
Outliers are defined as any values:
Below: Q1 - 1.5 × IQR
Above: Q3 + 1.5 × IQR

In [ ]:
Q1 = filtered_CO2_df["emissions_avoided"].quantile(0.25)
Q3 = filtered_CO2_df["emissions_avoided"].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
Q1

In [ ]:
# calculate the outliers

# Compute Q1, Q3, and IQR
Q1 = filtered_df["emissions_avoided"].quantile(0.25)
Q3 = filtered_df["emissions_avoided"].quantile(0.75)
IQR = Q3 - Q1

# Define outlier thresholds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = filtered_df[(filtered_df["emissions_avoided"] < lower_bound) | (filtered_df["emissions_avoided"] > upper_bound)]

# Display outliers
print(outliers[["impstatus", "emissions_avoided"]])

In [ ]:
# Define a path to save visualizations

# Define relative path
relative_path_vis = Path('../assets/')
absolute_path_vis = relative_path_vis.resolve() # get absolute path

In [ ]:
sns.set_theme(style="white")

# Define color palette
palette = {"N": "salmon", "I": 'lightgreen'}

# Create the boxplot
plt.figure(figsize=(8, 6))
ax = sns.boxplot(data=filtered_CO2_df, x="impstatus", y="emissions_avoided", 
                 width=0.4, palette=palette, 
                 showfliers=False, 
                 medianprops={'color': 'black', 'linewidth': 1},
                 hue='impstatus')

# Format y-axis with commas
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))  # Comma formatting

# Adjust titles and labels
plt.xlabel("Status", fontsize=16)
plt.ylabel("Emissions Avoided (kg)", fontsize=16)
plt.xticks(ticks=[0, 1], labels=["Not Implemented", "Implemented"], fontsize=14)
plt.yticks(fontsize=14)
#plt.title("CO2 Emissions Avoided by Implementation Status", fontsize=18)

# Customize legend
#handles, labels = ax.get_legend_handles_labels()
#plt.legend(handles[:2], ["Not Implemented", "Implemented"], fontsize=14)

# Save the plot
plt.savefig(absolute_path_vis / "emissions_avoided_boxplot.png", format='png')

plt.show()


In [ ]:
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))

# Filter data
not_implemented = filtered_CO2_df[filtered_CO2_df["impstatus"] == "N"]
implemented = filtered_CO2_df[filtered_CO2_df["impstatus"] == "I"]

# Create boxplots separately with different properties
ax = sns.boxplot(data=not_implemented, x=["Not Implemented"]*len(not_implemented), 
                y="emissions_avoided", width=0.4, showfliers=False,
                boxprops={'edgecolor': '#C44E52', 'facecolor': 'r', 'alpha': 0.6},
                medianprops={'color': '#C44E52', 'linewidth': 2},
                whiskerprops={'color': '#C44E52', 'linewidth': 1.5},
                capprops={'color': '#C44E52', 'linewidth': 1.5})

sns.boxplot(data=implemented, x=["Implemented"]*len(implemented), 
            y="emissions_avoided", width=0.4, showfliers=False,
            boxprops={'edgecolor': '#376A3E', 'facecolor': '#376A3E', 'alpha': 0.6},
            medianprops={'color': '#376A3E', 'linewidth': 2},
            whiskerprops={'color': '#376A3E', 'linewidth': 1.5},
            capprops={'color': '#376A3E', 'linewidth': 1.5},
            ax=ax)

# Format y-axis with commas
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))

# Adjust titles and labels
plt.title('CO2 Reduction Comparison By Implementation Status', 
          fontsize=18, 
          pad=20)
plt.xlabel("Status", fontsize=16)
plt.ylabel("Emissions Avoided (kg)", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig(absolute_path_vis / "emissions_avoided_boxplot.png", format='png')
plt.show()

In [ ]:
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))

# Filter data
not_implemented = filtered_SO2_df[filtered_SO2_df["impstatus"] == "N"]
implemented = filtered_SO2_df[filtered_SO2_df["impstatus"] == "I"]

# Create boxplots separately with different properties
ax = sns.boxplot(data=not_implemented, x=["Not Implemented"]*len(not_implemented), 
                y="emissions_avoided", width=0.4, showfliers=False,
                boxprops={'edgecolor': '#C44E52', 'facecolor': 'r', 'alpha': 0.6},
                medianprops={'color': '#C44E52', 'linewidth': 2},
                whiskerprops={'color': '#C44E52', 'linewidth': 1.5},
                capprops={'color': '#C44E52', 'linewidth': 1.5})

sns.boxplot(data=implemented, x=["Implemented"]*len(implemented), 
            y="emissions_avoided", width=0.4, showfliers=False,
            boxprops={'edgecolor': '#376A3E', 'facecolor': '#376A3E', 'alpha': 0.6},
            medianprops={'color': '#376A3E', 'linewidth': 2},
            whiskerprops={'color': '#376A3E', 'linewidth': 1.5},
            capprops={'color': '#376A3E', 'linewidth': 1.5},
            ax=ax)

# Format y-axis with commas
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.001f}'))

# Adjust titles and labels
plt.title('SO2 Reduction Comparison By Implementation Status', 
          fontsize=18, 
          pad=20)
plt.xlabel("Status", fontsize=16)
plt.ylabel("Emissions Avoided (kg)", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig(absolute_path_vis / "emissions_avoided_boxplot.png", format='png')
plt.show()

In [ ]:
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))

# Filter data
not_implemented = filtered_NOx_df[filtered_NOx_df["impstatus"] == "N"]
implemented = filtered_NOx_df[filtered_NOx_df["impstatus"] == "I"]

# Create boxplots separately with different properties
ax = sns.boxplot(data=not_implemented, x=["Not Implemented"]*len(not_implemented), 
                y="emissions_avoided", width=0.4, showfliers=False,
                boxprops={'edgecolor': '#C44E52', 'facecolor': 'r', 'alpha': 0.6},
                medianprops={'color': '#C44E52', 'linewidth': 2},
                whiskerprops={'color': '#C44E52', 'linewidth': 1.5},
                capprops={'color': '#C44E52', 'linewidth': 1.5})

sns.boxplot(data=implemented, x=["Implemented"]*len(implemented), 
            y="emissions_avoided", width=0.4, showfliers=False,
            boxprops={'edgecolor': '#376A3E', 'facecolor': '#376A3E', 'alpha': 0.6},
            medianprops={'color': '#376A3E', 'linewidth': 2},
            whiskerprops={'color': '#376A3E', 'linewidth': 1.5},
            capprops={'color': '#376A3E', 'linewidth': 1.5},
            ax=ax)

# Format y-axis with commas
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))

# Adjust titles and labels
plt.title('NOx Reduction Comparison By Implementation Status', 
          fontsize=18, 
          pad=20)
plt.xlabel("Status", fontsize=16)
plt.ylabel("Emissions Avoided (kg)", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig(absolute_path_vis / "emissions_avoided_boxplot.png", format='png')
plt.show()

In [ ]:
filtered_other_fuels_df

In [ ]:
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))

# Filter data
not_implemented = filtered_other_fuels_df[filtered_other_fuels_df["impstatus"] == "N"]
implemented = filtered_other_fuels_df[filtered_other_fuels_df["impstatus"] == "I"]

# Create boxplots separately with different properties
ax = sns.boxplot(
    data=not_implemented, 
    #x=["Not Implemented"]*len(not_implemented), 
    x="fuel_type",
    hue="impstatus",
    y="conserved", 
    width=0.4, showfliers=False,
    boxprops={'edgecolor': '#C44E52', 'facecolor': 'r', 'alpha': 0.6},
    medianprops={'color': '#C44E52', 'linewidth': 2},
    whiskerprops={'color': '#C44E52', 'linewidth': 1.5},
    capprops={'color': '#C44E52', 'linewidth': 1.5})
sns.boxplot(
    data=implemented, 
    #x=["Implemented"]*len(implemented), 
    x="fuel_type",
    hue="impstatus",
    y="conserved", 
    width=0.4, 
    showfliers=False,
    boxprops={'edgecolor': '#376A3E', 'facecolor': '#376A3E', 'alpha': 0.6},
    medianprops={'color': '#376A3E', 'linewidth': 2},
    whiskerprops={'color': '#376A3E', 'linewidth': 1.5},
    capprops={'color': '#376A3E', 'linewidth': 1.5},
    ax=ax)

# Format y-axis with commas
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))

# Adjust titles and labels
plt.title('Other Fuels Savings Distribution By Implementation Status', 
          fontsize=18, 
          pad=20)
plt.xlabel("Status", fontsize=16)
plt.ylabel("Energy Saved (MMBtu)", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig(absolute_path_vis / "emissions_avoided_boxplot.png", format='png')
plt.show()

In [ ]:
filtered_EC_df

In [ ]:
sns.set_theme(style="white")
plt.figure(figsize=(8, 6))

# Filter data
not_implemented = filtered_EC_df[filtered_EC_df["impstatus"] == "N"]
implemented = filtered_EC_df[filtered_EC_df["impstatus"] == "I"]

# Create boxplots separately with different properties
ax = sns.boxplot(data=not_implemented, x=["Not Implemented"]*len(not_implemented), 
                y="conserved", width=0.4, showfliers=False,
                boxprops={'edgecolor': '#C44E52', 'facecolor': 'r', 'alpha': 0.6},
                medianprops={'color': '#C44E52', 'linewidth': 2},
                whiskerprops={'color': '#C44E52', 'linewidth': 1.5},
                capprops={'color': '#C44E52', 'linewidth': 1.5})

sns.boxplot(data=implemented, x=["Implemented"]*len(implemented), 
            y="conserved", width=0.4, showfliers=False,
            boxprops={'edgecolor': '#376A3E', 'facecolor': '#376A3E', 'alpha': 0.6},
            medianprops={'color': '#376A3E', 'linewidth': 2},
            whiskerprops={'color': '#376A3E', 'linewidth': 1.5},
            capprops={'color': '#376A3E', 'linewidth': 1.5},
            ax=ax)

# Format y-axis with commas
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))

# Adjust titles and labels
plt.title('Electricity Savings Distribution By Implementation Status', 
          fontsize=18, 
          pad=20)
plt.xlabel("Status", fontsize=16)
plt.ylabel("Energy Saved (kWh)", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.savefig(absolute_path_vis / "emissions_avoided_boxplot.png", format='png')
plt.show()

In [ ]:
filtered_df.columns

In [ ]:
# manually update missing PPI
filtered_df[]

In [ ]:
concise_html = filtered_df[relevant_columns].to_html(index=False, classes='table table-striped')

# Add a simple style override to enforce a light background
light_mode_css = """
<style>
    .table {
        background-color: white !important;
        color: black !important;
    }
    .table-striped tbody tr:nth-of-type(odd) {
        background-color: #f9f9f9 !important;
    }
</style>
"""

# Generate a clean HTML table that can be copied to slides
html_table = filtered_df.to_html(index=False, classes='table table-striped')

# For a more concise table, select only the most relevant columns
# For example:
relevant_columns = ['superid', 'arc2', 'naics', 'state', 'fy', 'impstatus', 'impcost',
       'ref_year_impcost', 'source_rank', 'sourccode', 'conserved', 
       'emission_type','emissions_avoided', 'emission_factor', 'emission_factor_units'
       
       ]
concise_html = filtered_df[relevant_columns].to_html(index=False, classes='table table-striped')

# Or for an even cleaner format for presentations
from IPython.display import HTML
HTML(filtered_df[relevant_columns].to_html(index=False, classes='table table-striped'))

HTML(filtered_df[relevant_columns].to_html(index=False, classes='table table-striped table-light'))

# Combine the CSS and table
HTML(light_mode_css + concise_html)

In [ ]:
filtered_df[relevant_columns].to_csv("../data/intermediate_data/iac_integrated_mockupv2.csv", index=False)